# Insignificant Data Clean
- 以天为单位进行数据清洗
- 去除在同一地点的中段数据

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mat
import matplotlib.cm as cm
import os
import datetime
import time
from tqdm import tqdm
import plotly.express as px
import plotly.io as pio
import math

import sys

from utils import myplot
from utils import utils


# 设置plotly默认主题
pio.templates.default = 'plotly_white'

mat.rcParams['font.family'] = 'SimHei'
mat.rcParams['font.sans-serif'] = 'SimHei'

import warnings
warnings.filterwarnings('ignore')

from collections import namedtuple

from importlib import reload

import random

## 读取数据->打上date标签->初步清理->保存

In [2]:
df_read = pd.read_csv(os.getcwd()+'/wifi_track_data/dacang/track_data/dacang_track_data_1_1214.csv')
df = df_read.copy()
df_read

,a,r,t,m
0,22,-43,2022-07-05 23:49:36,"48,74,38,155,214,98"
1,22,-35,2022-07-05 23:49:36,"252,107,240,89,142,53"
2,22,-42,2022-07-05 23:49:36,"164,125,159,153,152,106"
3,22,-38,2022-07-05 23:49:47,"252,107,240,89,142,53"
4,22,-42,2022-07-05 23:49:47,"164,125,159,153,152,106"
...,...,...,...,...
1916096,66,-49,2022-08-07 23:44:30,"88,102,186,101,140,144"
1916097,66,-49,2022-08-07 23:46:54,"88,102,186,101,140,144"
1916098,66,-48,2022-08-07 23:48:38,"88,102,186,101,140,144"
1916099,66,-49,2022-08-07 23:51:14,"88,102,186,101,140,144"


In [6]:
#label each mac with date
pbar = tqdm(total=len(df))
df['oriMac'] = df['m']
df.t = pd.to_datetime(df.t)
df['calender'] = df.t.apply(lambda x : str(x.month)+'.'+str(x.day))
for index,row in df.iterrows():
    df.loc[index,'m'] = str(row['calender'])+'-'+row.m
    pbar.update(1)
pbar.close()
print("mac length:",len(df.m.unique()))
df

100%|██████████| 1916101/1916101 [01:35<00:00, 20156.77it/s]


mac length: 31798


,a,r,t,m,oriMac,calender
0,22,-43,2022-07-05 23:49:36,"7.5-48,74,38,155,214,98","48,74,38,155,214,98",7.5
1,22,-35,2022-07-05 23:49:36,"7.5-252,107,240,89,142,53","252,107,240,89,142,53",7.5
2,22,-42,2022-07-05 23:49:36,"7.5-164,125,159,153,152,106","164,125,159,153,152,106",7.5
3,22,-38,2022-07-05 23:49:47,"7.5-252,107,240,89,142,53","252,107,240,89,142,53",7.5
4,22,-42,2022-07-05 23:49:47,"7.5-164,125,159,153,152,106","164,125,159,153,152,106",7.5
...,...,...,...,...,...,...
1916096,66,-49,2022-08-07 23:44:30,"8.7-88,102,186,101,140,144","88,102,186,101,140,144",8.7
1916097,66,-49,2022-08-07 23:46:54,"8.7-88,102,186,101,140,144","88,102,186,101,140,144",8.7
1916098,66,-48,2022-08-07 23:48:38,"8.7-88,102,186,101,140,144","88,102,186,101,140,144",8.7
1916099,66,-49,2022-08-07 23:51:14,"8.7-88,102,186,101,140,144","88,102,186,101,140,144",8.7


In [7]:
#保存
df.to_csv(os.getcwd()+"/wifi_track_data/dacang/temp_data/dacang_track_data_2_dateLabeled_1214.csv",index=False)

## 以天为单位清理异常mac

In [4]:
df = pd.read_csv(os.getcwd()+"/wifi_track_data/dacang/temp_data/dacang_track_data_2_dateLabeled_1214.csv")
df.t = pd.to_datetime(df.t)
df = df.sort_values(by=['t']).reset_index(drop=True)
df

,a,r,t,m,oriMac,calender
0,22,-34,2022-07-05 16:51:34,"7.5-48,74,38,133,29,213","48,74,38,133,29,213",7.5
1,22,-44,2022-07-05 16:51:34,"7.5-48,74,38,155,214,98","48,74,38,155,214,98",7.5
2,22,-35,2022-07-05 16:51:34,"7.5-20,178,229,137,47,142","20,178,229,137,47,142",7.5
3,22,-85,2022-07-05 16:51:34,"7.5-172,48,74,38,155,214","172,48,74,38,155,214",7.5
4,22,-13,2022-07-05 16:51:34,"7.5-124,72,127,16,178,1","124,72,127,16,178,1",7.5
...,...,...,...,...,...,...
1916096,5,-49,2022-08-07 23:52:41,"8.7-252,4,28,35,119,153","252,4,28,35,119,153",8.7
1916097,5,-37,2022-08-07 23:52:41,"8.7-32,50,51,210,96,61","32,50,51,210,96,61",8.7
1916098,32,-32,2022-08-07 23:52:49,"8.7-156,95,90,12,124,73","156,95,90,12,124,73",8.7
1916099,32,-36,2022-08-07 23:52:49,"8.7-124,161,119,201,50,40","124,161,119,201,50,40",8.7


In [5]:
#删除打上标签后只出现过一次的mac
df = df[df.duplicated('m',keep=False)]
df

,a,r,t,m,oriMac,calender
0,22,-34,2022-07-05 16:51:34,"7.5-48,74,38,133,29,213","48,74,38,133,29,213",7.5
1,22,-44,2022-07-05 16:51:34,"7.5-48,74,38,155,214,98","48,74,38,155,214,98",7.5
2,22,-35,2022-07-05 16:51:34,"7.5-20,178,229,137,47,142","20,178,229,137,47,142",7.5
3,22,-85,2022-07-05 16:51:34,"7.5-172,48,74,38,155,214","172,48,74,38,155,214",7.5
6,22,-41,2022-07-05 16:51:34,"7.5-164,125,159,153,152,106","164,125,159,153,152,106",7.5
...,...,...,...,...,...,...
1916096,5,-49,2022-08-07 23:52:41,"8.7-252,4,28,35,119,153","252,4,28,35,119,153",8.7
1916097,5,-37,2022-08-07 23:52:41,"8.7-32,50,51,210,96,61","32,50,51,210,96,61",8.7
1916098,32,-32,2022-08-07 23:52:49,"8.7-156,95,90,12,124,73","156,95,90,12,124,73",8.7
1916099,32,-36,2022-08-07 23:52:49,"8.7-124,161,119,201,50,40","124,161,119,201,50,40",8.7


In [6]:
#获取每个mac一天中经过的探针数：a_count
count_list = df.groupby('m').a.value_counts()
df_count = count_list.to_frame().reset_index()
a_count = df_count.m.value_counts()
#获取每个mac一天中的数据量:s_count
count_list = df.m.value_counts()


mac_list = df.m.unique()
len(mac_list)

15615

In [15]:
def GetDuration(df_now):
    t1 = df_now.iloc[0].t
    t2 = df_now.iloc[len(df_now)-1].t
    duration =  t2-t1
    t = (t1 + duration/2)
    moment = t.hour + (t.minute/60)
    duration = duration.components.hours+(duration.components.minutes/60)
    return round(duration,2),round(moment,2)

macDur_dict = {}
macMoment_dict = {}

for mac in tqdm(mac_list):
    df_now = df[df.m == mac]
    duration,moment = GetDuration(df_now)
    macDur_dict.update({mac:duration})
    macMoment_dict.update({mac:moment})

#保存
dur_dict_str = str(macDur_dict)
mom_dict_str = str(macMoment_dict)
with open('wifi_track_data/dacang/temp_data/date_mac_duration_dict_2_1214.txt', 'w') as file:
   file.write(dur_dict_str)
with open('wifi_track_data/dacang/temp_data/date_mac_moment_dict_2_1214.txt', 'w') as file:
   file.write(mom_dict_str)

100%|██████████| 15615/15615 [14:43<00:00, 17.68it/s]


In [7]:
# #读取
with open('wifi_track_data/dacang/temp_data/date_mac_duration_dict_2_1214.txt', 'r') as file:
    dict_str = file.read()
    macDur_dict = eval(dict_str)
with open('wifi_track_data/dacang/temp_data/date_mac_moment_dict_2_1214.txt', 'r') as file:
    dict_str = file.read()
    macMoment_dict = eval(dict_str)
mac_list = list(macDur_dict.keys())
mac_dur_list = list(macDur_dict.values())
mac_moment_list = list(macMoment_dict.values())

In [8]:
#生成统计数值的dataframe
df_sta = pd.DataFrame({'M':[],'Dur':[],'A_count':[],'Count':[],'oriMac':[],'date':[]})
for i in range(len(mac_list)):
    df_sta = df_sta._append({'M':mac_list[i],
                             'Dur':mac_dur_list[i],
                             'Moment':mac_moment_list[i],
                             'A_count':a_count[mac_list[i]],
                             'Count':count_list[mac_list[i]],
                             'oriMac':mac_list[i].split('-')[1],
                             'date':mac_list[i].split('-')[0]},ignore_index=True)
df_sta.to_csv('wifi_track_data/dacang/temp_data/data_statistic_2_1214.csv',index=False)
df_sta

,M,Dur,A_count,Count,oriMac,date,Moment
0,"7.5-48,74,38,133,29,213",7.13,5.0,49.0,"48,74,38,133,29,213",7.5,20.42
1,"7.5-48,74,38,155,214,98",7.13,6.0,147.0,"48,74,38,155,214,98",7.5,20.42
2,"7.5-20,178,229,137,47,142",7.13,4.0,38.0,"20,178,229,137,47,142",7.5,20.42
3,"7.5-172,48,74,38,155,214",0.00,1.0,2.0,"172,48,74,38,155,214",7.5,16.85
4,"7.5-164,125,159,153,152,106",7.00,2.0,19.0,"164,125,159,153,152,106",7.5,20.35
...,...,...,...,...,...,...,...
15610,"8.7-164,201,57,205,13,69",0.07,1.0,6.0,"164,201,57,205,13,69",8.7,19.63
15611,"8.7-152,207,83,224,116,138",3.47,1.0,7.0,"152,207,83,224,116,138",8.7,21.48
15612,"8.7-216,138,220,202,211,84",1.93,1.0,8.0,"216,138,220,202,211,84",8.7,22.82
15613,"8.7-244,76,112,14,81,63",0.03,1.0,3.0,"244,76,112,14,81,63",8.7,21.88


In [21]:
df_sta = pd.read_csv('wifi_track_data/dacang/temp_data/data_statistic_2_1214.csv',index=False)
fig = px.histogram(df_sta.Count.to_list())
fig.show()

- MAC的数据量之间相差及其悬殊，先剔除数据量与预期相差过大的MAC
- 当MAC到达的探针数量<=2时，可视为：
  - 固定设备处于三个探针的信号覆盖范围中，予以剔除
  - 行人移动量过低，视为居家行为，其行为不受环境因素所影响，予以剔除

In [9]:
#--初步清理--#
#剔除一天中数据量小于10的mac
df_sta = df_sta[df_sta.Count.apply(lambda x : x>10)]
#剔除一天中持续时间小于半小时的mac
df_sta = df_sta[df_sta.Dur>0.5]
#剔除一天中接受探针数小于3的mac
df_sta = df_sta[df_sta.A_count>2]
df_sta = df_sta.reset_index().drop('index',axis = 1)
df_sta

,M,Dur,A_count,Count,oriMac,date,Moment
0,"7.5-48,74,38,133,29,213",7.13,5.0,49.0,"48,74,38,133,29,213",7.5,20.42
1,"7.5-48,74,38,155,214,98",7.13,6.0,147.0,"48,74,38,155,214,98",7.5,20.42
2,"7.5-20,178,229,137,47,142",7.13,4.0,38.0,"20,178,229,137,47,142",7.5,20.42
3,"7.5-4,207,140,11,128,186",7.12,3.0,15.0,"4,207,140,11,128,186",7.5,20.42
4,"7.5-208,151,254,40,222,169",7.10,3.0,11.0,"208,151,254,40,222,169",7.5,20.40
...,...,...,...,...,...,...,...
1955,"8.7-76,2,32,111,66,59",9.95,4.0,226.0,"76,2,32,111,66,59",8.7,12.60
1956,"8.7-56,251,20,50,26,70",12.08,4.0,88.0,"56,251,20,50,26,70",8.7,14.28
1957,"8.7-124,161,119,201,50,40",13.35,6.0,260.0,"124,161,119,201,50,40",8.7,17.20
1958,"8.7-144,173,247,200,14,93",8.18,4.0,44.0,"144,173,247,200,14,93",8.7,15.87


## 清除在一段时间内持续在同一地点的mac数据中段

仅保留头尾

部分居民设备信号强度大，导致其数据量明显大于别的mac。
将所有设备一直在同一时间段停留的数据只保留头尾两条

In [10]:
#!!根据date-Mac过滤主df
l = set(df_sta.M.unique().tolist())
df = df[df.m.apply(lambda x : x in l)].reset_index(drop=True)
df['mark'] = range(0,len(df))
mac_list = df.m.unique()
df.t = pd.to_datetime(df.t)

In [11]:
del_list = []
for mac in tqdm(mac_list):
    df_now = utils.GetDfNow(df,mac)
    del_list.extend(utils.GetRepeatTrack(df_now))
len(del_list)


100%|██████████| 1960/1960 [02:19<00:00, 14.08it/s]


567124

In [12]:
del_list = set(del_list)
df_filterd = df[df.mark.apply(lambda x:False if x in del_list else True)]
df_filterd.to_csv(os.getcwd()+"/wifi_track_data/dacang/track_data/dacang_track_data_2_1214.csv",index = False)
df_filterd

,a,r,t,m,oriMac,calender,mark
7,22,-37,2022-07-05 16:51:34,"7.5-48,74,38,133,29,213","48,74,38,133,29,213",7.5,7
8,22,-36,2022-07-05 16:51:34,"7.5-4,207,140,11,128,186","4,207,140,11,128,186",7.5,8
9,22,-51,2022-07-05 16:51:34,"7.5-208,151,254,40,222,169","208,151,254,40,222,169",7.5,9
11,22,-44,2022-07-05 16:51:34,"7.5-48,74,38,155,214,98","48,74,38,155,214,98",7.5,11
12,88,-44,2022-07-05 20:55:22,"7.5-48,74,38,133,29,213","48,74,38,133,29,213",7.5,12
...,...,...,...,...,...,...,...
1493588,5,-53,2022-08-07 23:52:28,"8.7-252,4,28,35,119,153","252,4,28,35,119,153",8.7,1493588
1493589,5,-49,2022-08-07 23:52:41,"8.7-252,4,28,35,119,153","252,4,28,35,119,153",8.7,1493589
1493590,5,-37,2022-08-07 23:52:41,"8.7-32,50,51,210,96,61","32,50,51,210,96,61",8.7,1493590
1493591,32,-32,2022-08-07 23:52:49,"8.7-156,95,90,12,124,73","156,95,90,12,124,73",8.7,1493591
